<a href="https://colab.research.google.com/github/Pedro-Boechat/trainee/blob/main/periodo4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.decomposition import PCA
import time

# Binário

In [227]:
df = pd.read_csv('/content/creditcard.csv')

In [228]:
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df.isna().sum()

In [229]:
df = df.dropna()

In [230]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [231]:
df = df.drop(columns='Time')

In [232]:
colunas = df.columns

## Normalizar

In [207]:
for i in colunas:
  df[i] = (df[i] - df[i].min()) / (df[i].max() - df[i].min())

In [208]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,0.510600,0.252484,0.680908,0.371591,0.635591,0.446084,0.434392,0.737173,0.655066,0.594863,0.582942,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0.0
1,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,0.505267,0.381188,0.744342,0.486190,0.641219,0.383840,0.464105,0.727794,0.640681,0.551930,0.579530,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0.0
2,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,0.513018,0.322422,0.706683,0.503854,0.640473,0.511697,0.357443,0.763381,0.644945,0.386683,0.585855,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0.0
3,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,0.507585,0.271817,0.710910,0.487635,0.636372,0.289124,0.415653,0.711253,0.788492,0.467058,0.578050,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0.0
4,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,0.524303,0.236355,0.724477,0.552509,0.608406,0.349419,0.434995,0.724243,0.650665,0.626060,0.584615,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0.0


## Verificar balanceamento

In [233]:
from sklearn.utils import resample
# Separar a maioria e minoria
df_majority = df[df['Class']==0]
df_minority = df[df['Class']==1]
# Downsample
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    
                                 n_samples=492) # Colocar aqui o downsample que quiser
# Combinar a maioria downsampled com a minoria
df = pd.concat([df_majority_downsampled, df_minority])

## Meu código

In [234]:
from sklearn.model_selection import train_test_split

X = df[colunas.drop('Class')]
Y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)


In [211]:
X_train.shape

(787, 29)

In [235]:
X_test = np.array(X_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [263]:
n_samples, n_features = X_train.shape
weights1 = np.random.rand(n_samples)
weights2 = np.random.rand(10)
hneurons = np.zeros(10)
bias1 = np.random.rand()
bias2 = np.random.rand()


In [237]:
def activation1(x): #Relu
  return x * (x > 0)

def dact1(x):
    return 1. * (x > 0)

def activation2(x): # ACTIVATION 2 é sigmoide
  return 1/(1 + np.exp(-x))


def dact2(x):
  return activation2(x) * (1.0 - activation2(x))


In [262]:
activation2(z2)

0.0008562779733391644

In [265]:
lista = []
predictions = []
iter = 200
learning_rate = 0.8
for _ in range(iter): # Para cada época
  predictions = []
  for i in range(len(X_train)): # Para cada X_train


    for k in range(len(hneurons)): # Para cada neuronio oculto
      lista = [] # Soma pesada de todos os neuronios de entrada
      for j in range(len(X_train[i])): # Para cada feature
        forward = (X_train[i][j] * weights1[j]) + bias1 # Pesa cada neuronio
        lista.append(forward) # Joga na lista da soma pesada

      z1 = np.sum(lista) # Z é a soma pesada
      ativada = activation2(z1) # Ativando Z com sigmoide
      hneurons[k] = ativada  # Cada neuronio oculto recebendo um valor
    soma = [] # Mesmo papel do "lista" anterior, mas agora da camada oculta pro output
    for p in range(len(hneurons)): # Para cada neuronio oculto
      forward = hneurons[p] * weights2[p] + bias2 # Pesa cada neuronio
      soma.append(forward) # Joga na lista da soma pesada
    z2 = np.sum(soma) # Faz a soma
    ativada_output = activation2(z2) # Ativa Z2 com a sigmoide
    if ativada_output >= 0.5: # Se o resultado for maior que 0,5: 1. Se nao, 0
      predicao = 1
    else:
      predicao = 0
  
    predictions.append(predicao) # Joga essa predicao numa lista com as predicoes dessa epoca

    cost = []

    custo = np.square(predicao - y_train[i]) # Custo
    cost.append(custo) # Essa linha e a proxima sao desnecessarias. Escrevi quando tava idiota mas nao muda nada
    custo_total = np.sum(cost)


### COMEÇANDO O BACKWARD. VERIFICAR SE OS CALCULOS DE DERIVADAS ESTAO CERTOS PQ EU NAO TENHO CTZ######


    for n in range(len(hneurons)): # Para cada neuronio oculto
      dw2 = (2 * (predicao - y_train[i]) * dact2(z2) * (hneurons[n])) # Derivada de cada um dos pesos 2
      weights2[n] = weights2[n] - learning_rate * dw2 # Atualizando um peso 2 de cada vez


    for g in range(len(X_train[i])): # Para cada neuronio de entrada
      dw1 = 2 * (predicao - y_train[i]) * dact2(z2) * sum(weights2) * dact2(z1) * (X_train[i][g]) # Derivada de cada um dos pesos 1
      weights1[g] = weights1[g] - learning_rate * dw1 # Atualizando um peso 1 de cada vez

    db2 = 2 *(predicao - y_train[i]) * dact2(z2) # Derivada do bias2
    db1 = 2 * (predicao - y_train[i]) * dact2(z2) * sum(weights2) * dact1(z1) # Derivada do bias 1


    bias1 -= learning_rate * db2 #Atualizando bias 1
    bias2 -= learning_rate * db2 #Atualizando bias2



In [261]:
hneurons[0] * weights2[0]

-1.238202150475155

In [260]:
hneurons[0]

1.0

In [259]:
activation2(z2)

0.0008562779733391644

In [258]:
X_train[0][0] * weights1[0]

-0.24155554695983902

In [268]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

confusion_matrix(predictions, y_train)

array([[238, 224],
       [149, 176]])

In [24]:
1/(1 + np.exp(-z))

0.5

In [35]:
weights2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [29]:
len(hneurons)

10

In [26]:
print(len(derw2))

830


In [27]:
print(sum(derw2))

5724.690429230827


## Daqui pra baixo são os códigos copiados ruins

In [ ]:
weights1 = []
weights2 = []
bias1 = 0
bias2 = 0
hidden = np.zeros(10)

def forward():
  for _ in range(iter):
    for idx, x_i in enumerate(X_train):
      forward = np.matmul(X_train,weights1) + bias1
      forward = activation1(forward)
      forward = np.matmul(forward, weights2) + bias2
      pred = activation2(forward)

      predictions = np.append(predictions, pred)
    
def backward():
  for i in range


Primeiro teste: Perceptron classification accuracy 0.9985955549313578

0.9982

array([[56831,    64],
       [   16,    51]])

0.76119

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

confusion_matrix(predictions, y_test)

array([[56831,    64],
       [   16,    51]])

In [ ]:
def relu(z):
    return x * (x > 0)
def forward_propagation(X, parameters):
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    cache = {"Z1": Z1,"A1": A1,"Z2": Z2,"A2": A2}
    
    return A2, cache
    

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    #number of training example
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
   
    dZ2 = A2-Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2,"db2": db2}
    
    return grads

In [ ]:
from scipy import stats

pred = np.array([])
k = 5
tam = len(distancias)
for i in range(tam):
  idxkneigh = distancias[i].argsort()[:k] # Pega os indices das k menores distancias
  vizinhos = [y_train[idxkneigh[j]] for j in range(k)] # Busca qual a classificação desses indices: red ou não
  mode_info = stats.mode(vizinhos) # Pega a moda das classificações dos KNN
  pred = np.append(pred, mode_info[0]) # Prediz

In [ ]:
acertos = 0
for i in range(tam):
  if pred[i] == y_test[i]:
    acertos += 1

print(acertos)

629


In [ ]:
acertos/(tam)

0.9828125

99% Precisão normalizado e com PCA

92% Precisão sem normalizar e sem PCA

98% Precisão normalizando e sem PCA